# **Домашняя работа**
1) Создать консольную программу-парсер, с выводом прогноза погоды. Дать
возможность пользователю получить прогноз погоды в его локации ( по
умолчанию) и в выбраной локации, на определенную пользователем дату.
Можно реализовать, как консольную программу, так и веб страницу.
Используемые инструменты: requests, beatifulsoup, остальное по желанию.
На выбор можно спарсить страницу, либо же использовать какой-либо API. 

In [65]:
#TODO :
#Check our ip address. Find out our location and display correct name place and temp on this location 

import requests
import re
from bs4 import BeautifulSoup 


headers = {
    'content-type': 'text',
    'Connection': 'keep-alive',
    'Search-Version': 'v3',
    'Accept': 'application/json',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}
result = requests.get("https://iplocation.com/", headers=headers, allow_redirects=True).text # Get our ip address and our location info


#print(result)
soup = BeautifulSoup(result, 'html.parser')

location_list = soup.find_all('span') #Ok. We use BeautifulSoup here)
script_list = soup.find_all('script') #and here.
location_list = list(location_list)
script_list = list(script_list)


# Usual regex now. Find our location
location_list = " ".join(map(str, location_list))
country_pattern = ('class=\"country_name\">([\w\s]+)</span>')
state_pattern = ('class=\"region_name\">([\w\s]+)</span')

script_list = " ".join(map(str, script_list))
latitude_longitude_pattern = ("([\d\.]+), ([\d\-\.]+)")


country_name = re.findall(country_pattern, location_list)
state_name = re.findall(state_pattern, location_list)
latitude_longitude =  re.findall(latitude_longitude_pattern, script_list)

print(f'country_name:{country_name[0]}, state_name:{state_name[0]}')
print(f'latitude:{latitude_longitude[0][0]}  longitude:{latitude_longitude[0][1]}')
# Ok. Let's find out the weather in country_name and state_name location.


url_weather_request = f"https://forecast.weather.gov/MapClick.php?lon={latitude_longitude[0][1]}&lat={latitude_longitude[0][0]}"
weather_response = requests.get(url_weather_request, headers=headers, allow_redirects=True).text

place_pattern = ("href=\"https://www.weather.gov/lwx\"\stitle=\"([\w,/\.\s]+)")
local_place_pattern = ("&local_place=([\w\s]+)&")
day_temp_f = ("myforecast-current-lrg\">([\w&;]+)<")
day_temp_c = ("myforecast-current-sm\">([\w&;]+)<")


place = re.findall(place_pattern, weather_response)
place = str(place[0])
#place.replace("&deg;","")
print(place)

local_place = re.findall(local_place_pattern, weather_response)
local_place = str(local_place[0])
print(local_place)

temp_f = re.findall(day_temp_f, weather_response)
temp_f = str(temp_f[0]).replace("&deg;","")
print(temp_f)

temp_c = re.findall(day_temp_c, weather_response)
temp_c = str(temp_c[0]).replace("&deg;","")
print(temp_c)



country_name:United States, state_name:Virginia
latitude:38.6582  longitude:-77.2497
Baltimore, MD/Washington, D.C.
2 Miles SSE Occoquan VA
68F
20C
